# Projeto 01: Reconhecimento de Dígitos

### João Victor Soares Silva | Mat: 20210027300
### Yhasmim de Souza Tigre | Mat: 20210026966 

## Pré-Processamento

In [19]:
import numpy as np

In [20]:
N = 28 #Número de linhas e colunas das matrizes das imagens
P_MAX = 255 #Número máximo de um pixel

In [21]:
'''
Aqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.
Acho bom usar pra estudar pra apresentação, então deixei aqui.
'''

# teste = [
# 1, 2, 3, 4,
# 5, 6, 7, 8,
# 9, 10, 11, 12,
# 13, 14, 15, 16]

# lin = 4

# #Vertical
# print('Vertical')
# for i in range(lin):

#     for j in range(int(lin/2)):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*i) + (lin - j - 1)]} = {teste[(lin*i) + j] + teste[(lin*i) + (lin - j - 1)]}')

# #Horizontal
# print('\nHorizontal')
# for i in range(int(lin/2)):
    
#     for j in range(lin):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*(lin - i - 1)) + j]} = {teste[(lin*i) + j] + teste[(lin*(lin - i - 1)) + j]}')

'\nAqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.\nAcho bom usar pra estudar pra apresentação, então deixei aqui.\n'

In [22]:
def reduzir (nome : str, arq : str) -> None:
    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas
    lista_y = []
    lista_intensidade = []
    lista_simetria_vertical = []
    lista_simetria_horizontal = []
    lista_simetria = []

    for i in range (len(aux)): #Salvando as labels
        lista_y.append (int(aux[i][0]))
    
    aux = np.delete (aux, 0, 1) #Removendo as labels
    
    for i in range (len(aux)): #Calculando as intensidades
        soma = 0

        for j in range (len(aux[0])):
            soma += int(aux[i][j])
        
        lista_intensidade.append(soma/P_MAX)
    
    for imagem in aux: #Simetria vertical
        soma = 0

        for i in range(N):
            for j in range(int(N/2)):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[(N*i) + (N - j - 1)]))

        lista_simetria_vertical.append(soma/P_MAX)

    for imagem in aux: #Simetria horizontal
        soma = 0

        for i in range(int(N/2)):
            for j in range(N):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[N * (N - i - 1) + j]))

        lista_simetria_horizontal.append(soma/P_MAX)
    
    for i in range (len(aux)): #Somando as simetrias
        lista_simetria.append(lista_simetria_vertical[i] + lista_simetria_horizontal[i])

    #Criando e preenchendo o novo .csv
    dados_redux = open (f'{nome}.csv', 'w')

    dados_redux.write ('label;intensidade;simetria\n')
    for i in range (len(aux)):
        dados_redux.write (f'{lista_y[i]};{lista_intensidade[i]};{lista_simetria[i]}\n')
    
    dados_redux.close()

    return

In [23]:
reduzir(nome = 'train_redu', arq = 'train.csv')
reduzir(nome = 'test_redu', arq = 'test.csv')

## Algoritmos

### Perceptron Learning Algorithm (PLA)

In [24]:
class PLA ():
    def __init__(self, arq : str, w_lista = []) -> None:
        '''
        Entradas : arq - .csv dos dados, contendo suas características e rotulação. 
        '''
        self.xy_lista = []
        self.w_lista = w_lista

        aux = np.loadtxt (arq, delimiter = ';', dtype = str)
        aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas

        for linha in aux:
            self.xy_lista.append([[1, linha[1], linha[2]], linha[0]]) #O 1 será usado para a multiplicação com o bias posteriormente
        
        if (len(w_lista) == 0):
            for i in range (3): #Já que temos 2 features + o bias
                self.w_lista.append(0) #Os pesos inicializarão em 0, caso não existam

    def Acuracia (self, w_lista : list) -> float:
        soma_PCC = 0

        for i in range (len(self.xy_lista)):
            aux = np.sign (np.sum(np.multiply(w_lista, self.xy_lista[i][0])))

            if (aux == self.xy_lista[i][1]):
                soma_PCC += 1
    
        return (soma_PCC/len(self.xy_lista))

    def ConstrutorLista_PCI (self, pontos : list) -> list:
        lista_PCI = []

        for i in range (len(pontos)):
            aux = np.sign (np.sum(np.multiply(self.w_lista, pontos[i][0])))

            if (aux != pontos[i][1]):
                lista_PCI.append(pontos[i])
        
        return lista_PCI

    def ExecutarPLA (self, n_int = 100) -> None:
        lista_PCI = self.xy_lista
        w_otimo = self.w_lista

        i = 0
        while (len(lista_PCI) > 0 or i < n_int):
            ale_index = np.random.randint(0, len(lista_PCI)) #index aleatório
            ponto = lista_PCI[ale_index]

            aux = np.multiply(ponto[0], ponto[1]) #x * y
            w_novo = np.add(self.w_lista, aux)

            if (self.Acuracia(w_lista = w_otimo) < self.Acuracia(w_lista = w_novo)):
                w_otimo = w_novo

            self.w_lista = w_novo

            lista_PCI = self.ConstrutorLista_PCI (self, pontos = self.xy_lista)
            i += 1
        
        self.w_lista = w_otimo

        return


### Regressão Linear

### Regressão Logística

In [25]:
class Reg_Log ():
    def __init__ (self) -> None:
        pass

## Classificação dos Dígitos 1x5

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Classificação de Dígitos Completo

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Comparação dos Classificadores

## Implementações Avançadas

### Weight-Decay para a Regressão Logística

### Definindo a Melhor Ordem